In [ ]:
!pip install --quiet litellm
!pip install --quiet crewai
!pip install --quiet crewai-tools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 55.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.2/233.2 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.8/131.8 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 26.6 MB/s eta 0:00:

In [ ]:
from crewai import Agent, Task, Crew, Process, LLM
from crewai_tools import SerperDevTool, ScrapeWebsiteTool
from crewai.tools import BaseTool
from litellm import completion
import os
from google.colab import userdata
from pydantic import BaseModel, Field
from typing import List, Optional

In [ ]:
google_api_key = userdata.get('GOOGLE_API_KEY')
os.environ["GEMINI_API_KEY"] = userdata.get('GOOGLE_API_KEY')

llm = LLM(
    model="gemini/gemini-1.5-flash",
    temperature=0.7,
)

In [ ]:
class Activity(BaseModel):
    name: str = Field(..., description="Name of the activity")
    location: str = Field(..., description="Location of the activity")
    description: str = Field(..., description="Description of the activity")
    date: str = Field(..., description="Date of the activity")
    cousine: str = Field(..., description="Cousine of the restaurant")
    why_its_suitable: str = Field(..., description="Why it's suitable for the traveler")
    reviews: Optional[List[str]] = Field(..., description="List of reviews")
    rating: Optional[float] = Field(..., description="Rating of the activity")

class DayPlan(BaseModel):
	date: str = Field(..., description="Date of the day")
	activities: List[Activity] = Field(..., description="List of activities")
	restaurants: List[str] = Field(..., description="List of restaurants")
	flight: Optional[str] = Field(None, description="Flight information")

class Itinerary(BaseModel):
  name: str = Field(..., description="Name of the itinerary, something funny")
  day_plans: List[DayPlan] = Field(..., description="List of day plans")
  hotel: str = Field(..., description="Hotel information")

In [ ]:
class SupriseTravelAgent():
  def personalized_activity_planner_agent(self):
    return Agent(
        role="Activity Planner",
        goal="Research and find cool things to do at the destination, including \
        activities and events that match the traveler's interests and age group",
        backstory=" You are skilled at creating personalized itineraries that cater\
         to the specific preferences and demographics of travelers.",
        tools=[SerperDevTool(), ScrapeWebsiteTool()],
        llm=llm
    )

  def restaurant_scout_agent(self):
    return Agent(
        role="Restaurant Scout",
        goal="Find highly-rated restaurants and dining experiences at the destination,\
        and recommend scenic locations and fun activities",
        backstory="As a food lover, you know the best spots in town for a delightful\
        culinary experience. You also have a knack for finding picturesque and \
        entertaining locations",
        tools=[SerperDevTool(), ScrapeWebsiteTool()],
        llm=llm
    )

  def itinerary_compiler_agent(self):
    return Agent(
        role="Itinerary Compiler",
        goal="Compile all researched information into a comprehensive day-by-day itinerary,\
        ensuring the integration of flights and hotel information",
        backstory=" With an eye for detail, you organize all the information into a\
        coherent and enjoyable travel plan.",
        tools=[SerperDevTool()],
        llm=llm
    )

In [ ]:
class SupriseTravelTask():
  def personalized_activity_planning_task(self, agent):
    return Task(
      description="Research and find cool things to do at {destination}. Focus \
      on activities and events that match the traveler's interests and age group.\
      Utilize internet search tools and recommendation engines to gather the information.\
      Traveler's information: \
      - origin: {origin}\
      - destination: {destination}\
      - age of the traveler: {age}\
      - hotel localtion: {hotel_location}\
      - flight infromation: {flight_information}\
      - how long is the trip: {trip_duration}",
      agent=agent,
      expected_output="A list of recommended activities and events for each day of the trip.\
      Each entry should include the activity name, location, a brief description, and\
      why it's suitable for the traveler. And potential reviews and ratings of the activities."
    )

  def restaurant_scenic_location_scout_task(self, agent):
    return Task(
      description="Find highly-rated restaurants and dining experiences at {destination}.\
      Recommend scenic locations and fun activities that align with the traveler's preferences.\
      Use internet search tools, restaurant review sites, and travel guides.\
      Make sure to find a variety of options to suit different tastes and budgets, and ratings for them.\
      Traveler's information:\
      - origin: {origin}\
      - destination: {destination}\
      - age of the traveler: {age}\
      - hotel localtion: {hotel_location}\
      - flight infromation: {flight_information}\
      - how long is the trip: {trip_duration}",
      agent=agent,
      expected_output=" A list of recommended restaurants, scenic locations, \
      and fun activities for each day of the trip. Each entry should include the \
      name, location (address), type of cuisine or activity, and a brief description\
      and ratings."
    )

  def itinerary_compilation_task(self, agent):
    return Task(
      description="Compile all researched information into a comprehensive day-by-day\
        itinerary for the trip to {destination}. Ensure the itinerary integrates flights,\
        hotel information, and all planned activities and dining experiences. Use text \
        formatting and document creation tools to organize the information.",
      agent=agent,
      expected_output="A detailed itinerary document, the itinerary should include a day-by-day\
      lan with flights, hotel details, activities, restaurants, and scenic locations.",
      output_json=Itinerary
    )

In [ ]:
class MyCustomTool(BaseTool):
  name: str = "Name of my tool"
  description: str = (
      "Clear description for what this tool is useful for, you agent will need this information to use it."
  )

  def _run(self, argument: str) -> str:
      # Implementation goes here
      return "this is an example of a tool output, ignore it and move along."

In [ ]:
agents = SupriseTravelAgent()
tasks = SupriseTravelTask()

personalized_activity_planner = agents.personalized_activity_planner_agent()
restaurant_scout = agents.restaurant_scout_agent()
itinerary_compiler = agents.itinerary_compiler_agent()

personalized_activity_planning_task = tasks.personalized_activity_planning_task(personalized_activity_planner)
restaurant_scenic_location_scout_task = tasks.restaurant_scenic_location_scout_task(restaurant_scout)
itinerary_compilation_task = tasks.itinerary_compilation_task(itinerary_compiler)

crew = Crew(
    agents=[personalized_activity_planner, restaurant_scout, itinerary_compiler],
    tasks=[personalized_activity_planning_task, restaurant_scenic_location_scout_task, itinerary_compilation_task],
    process=Process.sequential,
    verbose=True
)

inputs = {
        'origin': 'São Paulo, GRU',
        'destination': 'New York, JFK',
        'age': 31,
        'hotel_location': 'Brooklyn',
        'flight_information': 'GOL 1234, leaving at June 30th, 2024, 10:00',
        'trip_duration': '14 days'
    }

results = crew.kickoff(inputs=inputs)
print(results)

# Agent: Activity Planner
## Task: Research and find cool things to do at New York, JFK. Focus       on activities and events that match the traveler's interests and age group.      Utilize internet search tools and recommendation engines to gather the information.      Traveler's information:       - origin: São Paulo, GRU      - destination: New York, JFK      - age of the traveler: 31      - hotel localtion: Brooklyn      - flight infromation: GOL 1234, leaving at June 30th, 2024, 10:00      - how long is the trip: 14 days


# Agent: Activity Planner
## Thought: ```tool_code
Thought: I need to find cool things to do in New York City for a 31-year-old traveler from São Paulo, considering their 14-day trip starting June 30th, 2024, and staying in Brooklyn.  I'll use search engines to find activities and events.
## Using tool: Search the internet with Serper
## Tool Input: 
"{\"search_query\": \"Things to do in New York City for 30-something travelers\"}"
## Tool Output: 

I encountere

In [ ]:
results.to_dict()

{'name': 'New York City 14-Day Itinerary',
 'day_plans': [{'date': 'Day 1',
   'activities': [{'name': 'Arrival & Brooklyn Exploration',
     'location': 'Brooklyn (Williamsburg)',
     'description': 'Arrive at JFK, travel to Brooklyn hotel. Explore your neighborhood!',
     'date': 'Day 1',
     'cousine': None,
     'why_its_suitable': 'Relaxing start after a long flight, allows for acclimatization.',
     'reviews': None,
     'rating': None}],
   'restaurants': ['Lilia (Italian)'],
   'flight': 'Arrival at JFK'},
  {'date': 'Day 2',
   'activities': [{'name': 'Statue of Liberty & Ellis Island',
     'location': 'Lower Manhattan',
     'description': 'Ferry to Liberty and Ellis Islands. Explore the history and iconic views.',
     'date': 'Day 2',
     'cousine': None,
     'why_its_suitable': 'Classic NYC experience, impressive historical significance.',
     'reviews': None,
     'rating': None}],
   'restaurants': ['Shake Shack (Near Battery Park)'],
   'flight': None},
  {'date